In [ ]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

In [ ]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("openai_key")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

In [ ]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")


In [ ]:
#system prompt that defines assistant behavior 

sys_msg= SystemMessage(
    content="You are a friendly assistant but answers user question.Be careful and concise."
)
def assistant(state:MessagesState)-> dict:
    """
    The assistant node - processes messages and generates response.
    """

    # combine system prompt with conversation history
    messages=[sys_msg] + state["messages"]

    # Get response from LLM
    response=llm.invoke(messages)

    # Return as state update
    return {"messages": [AIMessage(content=response.content)]}

print("✅ Assistant node defined")


In [ ]:
# create a stategraph with messagesState

builder= StateGraph(MessagesState)

#Add the assistant mode
builder.add_node("assistant",assistant)

#define the flow 
# START → assistant → END
builder.add_edge(START,"assistant")
builder.add_edge("assistant",END)

In [ ]:
#create a memory checkpointer (stores in memory)
memory=MemorySaver()

#compile the graph with memory
agent=builder.compile(checkpointer=memory)

In [ ]:
def interactive_chat():
    """
    Run an interactive chat session.
    Type 'exit' or 'quit' to stop.
    """
    print("\n" + "="*70)
    print("Interactive Chat Started")
    print("Type your message and press Enter. Type 'exit' to quit.")
    print("="*70 + "\n")

    thread_id = "interactive_session2"

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("\nGoodbye...")
            break
        
        if not user_input:
            continue
        
        #Get response
        result = agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"thread_id": thread_id}}
        )

        # Print response
        agent_message = result["messages"][-1]
        user_message = result['messages'][-2]
        print(f"\nYou: {user_message.content}")
        print(f"\nAgent: {agent_message.content}")


In [ ]:
interactive_chat()